In [ ]:
try:
    import torch
    print("successfully imported torch")
except ImportError:
    !! pip install torch torchvision torchaudio --user --index-url https://download.pytorch.org/whl/cpu
    print("completed installing torch")

In [ ]:
try:
    import numpy as np
    print("successfully imported numpy")
except ImportError:
    !! pip install numpy --user --upgrade
    print("completed installing numpy")

In [ ]:
try:
    from matplotlib import pyplot as plt
    print("successfully imported matplotlib")
except ImportError:
    !! pip install matplotlib --user --upgrade
    print("completed installing matplotlib")

In [ ]:
!! pip install aenet-gpr --user --upgrade
print("completed installing aenet-gpr")

! pip show aenet-gpr

In [ ]:
import os, sys, site
sys.path.append(site.USER_SITE)

import aenet_gpr
print(aenet_gpr.__version__)

# 1. Prepare training/test data

We are using train/test HSE06 data of Li/EC interface prepared in `aenet-gpr/example/3_Li-EC/`

In [ ]:
import os.path

! mkdir 3_Li-EC
    
if os.path.isfile("../example/3_Li-EC/train_set.zip"):
    ! unzip -oq ../example/3_Li-EC/train_set.zip -d ./3_Li-EC/
else:
    ! wget https://github.com/atomisticnet/aenet-gpr/raw/refs/heads/main/example/3_Li-EC/train_set.zip
    ! unzip -oq train_set.zip -d ./3_Li-EC/
    ! rm train_set.zip

In [ ]:
if os.path.isfile("../example/3_Li-EC/test_set.zip"):
    ! unzip -oq ../example/3_Li-EC/test_set.zip -d ./3_Li-EC/
else:
    ! wget https://github.com/atomisticnet/aenet-gpr/raw/refs/heads/main/example/3_Li-EC/test_set.zip
    ! unzip -oq test_set.zip -d ./3_Li-EC/
    ! rm test_set.zip

# 2. Train–Test–Augmentation execute

In addition to the reference data files, following aenet-gpr input file `train.in` is all you need to run aenet-gpr. 

Most of the contents are set to default parameters, which can also be deleted.

In [ ]:
if os.path.isfile("../example/3_Li-EC/train.in"):
    ! cp ../example/3_Li-EC/train.in .
else:
    ! wget https://github.com/atomisticnet/aenet-gpr/raw/refs/heads/main/example/3_Li-EC/train.in

In [ ]:
! cat train.in

In [ ]:
! python -m aenet_gpr train.in

# 3. Visualize the results

In [ ]:
energy_test_gpr = np.load("./energy_test_gpr.npy")
energy_test_reference = np.load("./energy_test_reference.npy")
force_test_gpr = np.load("./force_test_gpr.npy")
force_test_reference = np.load("./force_test_reference.npy")
uncertainty_test_gpr = np.load("./uncertainty_test_gpr.npy")

In [ ]:
diff = energy_test_reference - energy_test_gpr

print(diff.min(), diff.max())

In [ ]:
fig = plt.figure(figsize=(7, 6))
ax2 = fig.subplots()

font_x = {'size': 20, 'color': 'black'}
font_y = {'size': 20, 'color': 'black'}
font_tick = {'size': 16, 'color': 'black'}


# The second plot
counts, bins = np.histogram(diff, bins=30, range=(-0.5, 2.5))

ax2.hist(bins[:-1], bins, weights=counts, facecolor='dimgray', alpha=1.0, edgecolor='black')

ax2.set_xlabel("DFT $-$ GPR (eV)", fontdict=font_x)
ax2.set_xlim([-0.5, 1.0])
x_labels = [round(label, 1) for label in ax2.get_xticks()]
x_labels = [-0.5, 0.0, 0.5, 1.0]
ax2.set_xticks(x_labels)
ax2.set_xlim([-0.5, 1.0])
ax2.set_xticklabels(x_labels, fontdict=font_tick)

ax2.set_ylabel("Number of structures", fontdict=font_y)
ax2.set_ylim([0, 100])
y_labels = [int(label) for label in ax2.get_yticks()]
# y_labels = [0, 20, 40, 60, 80, 100]
ax2.set_yticks(y_labels)
ax2.set_ylim([0, 100])
ax2.set_yticklabels(y_labels, fontdict=font_tick)

[x.set_linewidth(1.5) for x in ax2.spines.values()]
ax2.tick_params(bottom=False, top=True, left=True, right=True)
ax2.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
ax2.tick_params(direction='in', length=8, width=1.5)

ax2.grid(True, which='both', linestyle=':', linewidth=0.7, alpha=0.7)

fig.tight_layout()
plt.show()